In [1]:
from rl_ood import *
path = Path.cwd()

# OOD environments instantiation

In [2]:
default_values = MOUNTAIN_CAR_VALUES
default_values

{'Gravity': 0.0025, 'Force': 0.001}

Each ood config differs from the defaut environment by only one attribute.

In [3]:
values = create_ood_values(default_values)
ood_configs = get_ood_configs(default_values, values)
len(ood_configs)


20

# Evaluate OOD impact

### Original environment

We train an agent on the original environment, and we evaluate its performances

In [4]:
env = instanciate_mountain_car(default_values)
path_agent = path / 'Agents' / 'trained_mountain_car'
#if os.path.exists(path_agent):
#    agent = DQN.load(path_agent)
#else:
try:
    agent = DQN.load(path/'Agents'/'trained_mountain_car', env=env)
except:
    agent = DQN('MlpPolicy', env, learning_rate=0.001, buffer_size=10000, learning_starts=50000, batch_size=128, tau=1.0, gamma=0.99, train_freq=4)
    agent.learn(1000000, progress_bar=True)
    agent.save(path_agent)
evaluate(env, agent, nb_episodes=10, render=True)

(-150.3, 18.56906028855526)

We evaluate this agent on several ood environments. We get the mean and std of the reward over 10 episodes.

In [5]:
ood_env = instanciate_mountain_car(ood_configs[-1])
evaluate(ood_env, agent, render=False)

In [ ]:
#print("number of ood values per attribute:", len(values['Gravity']))

### Impact of the ood on the reward

In [ ]:
default_values, values = get_mountain_car_values()
results = {}
std_results = {}

original_env = instanciate_mountain_car(default_values)
agent = A2C.load(path/'Agents'/'trained_mountain_car_10000', env=original_env)
original_result, original_std_result = evaluate(original_env, agent, render=False)

for config in tqdm(ood_configs):
    if config['change'] not in results:
        results[config['change']] = []
        std_results[config['change']] = []
    
    ood_env = instanciate_mountain_car(config)
    agent = A2C.load(path/'Agents'/'trained_mountain_car_10000', env=ood_env)
    mean_reward, std_reward = evaluate(ood_env, agent, nb_episodes=100)
    
    results[config['change']].append(mean_reward)
    std_results[config['change']].append(std_reward)
    #print(config, mean_reward)
    



In [ ]:
width

In [ ]:
for key in results:
    fig, ax = plt.subplots(figsize=(10, 4), dpi=200)
    ax.set_xscale('log')
    
    labels = copy(values[key])
    labels.append(default_values[key])
    labels = sorted(labels)
    
    width = np.array([2**i for i in range(-5, 0)] + [2**i for i in range(1, 5+1)]) #np.array(labels[1:]) - np.array(labels[:-1]) 
    width *= default_values[key]/3.0
    #print(width, np.array([width[-1]]))
    #width = np.concatenate((width, np.array([width[-1]])))/ 5
    #print(width)
    ax.bar(default_values[key], original_result, yerr=original_std_result, width=default_values[key]/3.0, label='Original')
    ax.bar(values[key], results[key], yerr=std_results[key], width=width, label='OOD')
    
    ax.set_xlabel(key)
    ax.set_ylabel('Mean episode duration')
    ax.set_title('Impact of OOD regarding '+key)
    ax.legend()
    """
    ax.bar(str(default_values[key]), original_result, width=width)
    labels = [str(x) for x in values[key]]
    ax.bar(labels, results[key], width=width)
    """
    

# Create OOD detector

The configuration ood detector is relative to its default environment

In [ ]:
default_values, values = get_mountain_car_values()
env = instanciate_mountain_car(default_values)
ood_detector = MartingaleOODDetector(env, verbose=False)
default_values

We have a low ood score on the delaut environment

In [ ]:
ood_detector.get_in_distrib_score()

It should be higher on ood environment

In [ ]:
ood_config = get_ood_configs(default_values, values)[-1]
ood_env = instanciate_mountain_car(ood_config)
ood_config

In [ ]:
ood_score = ood_detector.test_ood(ood_env, nb_steps=100)
ood_score

Let's compute the ood scores of the previously studies ood environments

In [ ]:
default_values, values = get_mountain_car_values()
mean_ood_scores = {}
std_ood_scores = {}

original_env = instanciate_mountain_car(default_values)
agent = A2C.load(path/'Agents'/'trained_mountain_car_10000', env=original_env)
ood_detector = MartingaleOODDetector(env)
in_distrib_score = ood_detector.get_in_distrib_score()

for config in tqdm(ood_configs):
    if config['change'] not in mean_ood_scores:
        mean_ood_scores[config['change']] = []
        std_ood_scores[config['change']] = []
    
    ood_env = instanciate_mountain_car(config)
    agent = A2C.load(path/'Agents'/'trained_mountain_car_10000', env=ood_env)
    list_scores = np.array([ood_detector.test_ood(ood_env, nb_steps=100) for _ in range(10)])
    ood_score = list_scores.mean()
    std_score = list_scores.std()

    #mean_reward, std_reward = evaluate(ood_env, agent, nb_episodes=10)

    mean_ood_scores[config['change']].append(ood_score)
    std_ood_scores[config['change']].append(std_score)
    #print(config, mean_reward)
    



We now realise a plot of the mean reward and ood scores of each environment. The ones of the delaut environment are in darker colour at the middle of the plot.

In [ ]:
mean_ood_scores

In [ ]:
mean_ood_scores['Gravity'] > 1.1

In [ ]:
{std_ood_scores[k] > 1.1 for k in std_ood_scores}

In [ ]:
{std_ood_scores[k][6] for k in std_ood_scores}

In [ ]:
from turtle import color
from numpy import inf


for key in tqdm(results):
    fig, ax = plt.subplots(figsize=(10, 4), dpi=200)
    ax.set_xscale('log')
    ax.set_yscale('log')
    
    labels = copy(values[key])
    labels.append(default_values[key])
    labels = sorted(labels)
    
    #mean_ood_scores[key] = np.nan_to_num(mean_ood_scores[key], copy=True, nan=0.0, posinf=max(mean_ood_scores[key]), neginf=0)
    #std_ood_scores[key] = np.nan_to_num(std_ood_scores[key], copy=True, nan=0.0, posinf=max(std_ood_scores[key]), neginf=0)

    default_width = default_values[key]/6.0
    width = np.array([2**i for i in range(-5, 0)] + [2**i for i in range(1, 5+1)]) #np.array(labels[1:]) - np.array(labels[:-1]) 
    width *= default_width
    #print(width, np.array([width[-1]]))
    #width = np.concatenate((width, np.array([width[-1]])))/ 5
    #print(width)

    ax.bar(default_values[key]- default_width/2, 1.0, width=default_width, label='Mean reward', color='darkgreen')
    ax.bar(values[key]- width/2, results[key]/original_result, width=width,  color='green')

    ax.bar(default_values[key]+default_width/2, 1.0, width=default_width, label='OOD score', color='darkorange')
    ax.bar(values[key]+ width/2, mean_ood_scores[key]/in_distrib_score, width=width, color='orange', yerr=std_ood_scores[key])
    
    ax.set_xlabel(key)
    ax.set_ylabel('Mean episode duration')
    ax.set_title('Impact of OOD regarding '+key)

    plt.legend() #prop={'size': 'medium'}, ncol=2)
   

We add as maning in distribution examples and there are ood examples

In [ ]:
original_env = instanciate_mountain_car(default_values)
agent = A2C.load(path/'Agents'/'trained_mountain_car_10000', env=original_env)

list_scores = []
for _ in trange(5*18):
    list_scores.append(np.array([ood_detector.test_ood(original_env, nb_steps=100) for _ in range(10)]).mean())

mean_ood_scores['None'] = list_scores

# Detection speed at a fixed threshold

In [ ]:
std_ood_scores['Gravity']

In [ ]:
original_env = instanciate_mountain_car(default_values)
list_scores = np.array([ood_detector.test_ood(original_env, nb_steps=1000) for _ in range(100)])
ood_score = list_scores.mean()
std_score = list_scores.std()

ood_score, std_score, threshold

In [ ]:
from rl_ood import *

In [ ]:
threshold = ood_score + 10*std_score

In [ ]:
np.array([ood_detector.test_ood(original_env, nb_steps=100) for _ in range(100)]) > threshold

In [ ]:
X_val[:20].shape

In [ ]:

class MartingaleOODDetector():
    def __init__(self, env: gym.Env, verbose=False, *args, **kwargs) -> None:

        self.verbose = verbose

        # training the model
        X_pred, y_pred = create_dataset(env, nb_steps=10000)
        self.pred_model = MultiOutputRegressor(KNeighborsRegressor()).fit(X_pred, y_pred)
        #self.conf_model = conf_model

        self.in_distrib_score = self.test_ood(env)

        if self.verbose:
            print("Anomaly score of the training distribution: ", self.in_distrib_score)

    def get_in_distrib_score(self):
        return self.in_distrib_score

    def test_ood(self, env, nb_steps=1000):
        """
        Compute the ood score
        """
        X_val, y_val = create_dataset(env, nb_steps)
        errors = np.abs((self.pred_model.predict(X_val) - y_val))

        if self.verbose:
            print("Absolute error")
            print("Mean: ", errors.mean())
            print("Std: ", errors.std())
            print()


        # Calibration of the ood detector
        pre_ood_score = martingale(compute_p_values(errors))   
        ood_score = nb_steps * np.log(1 + pre_ood_score) #/nb_steps
        #print("corrected score ", np.log10(ood_score)/nb_steps)
        return ood_score

    def stop_above_threshold(self, env, threshold, start_at=50, nb_steps=1000):
        """
        Compute the number of step for the ood score to go above the threshold
        """
        X_val, y_val = create_dataset(env, nb_steps)

        for step in range(start_at,nb_steps):
            errors = np.abs((self.pred_model.predict(X_val[:step]) - y_val[:step]))

            if self.verbose:
                print("Absolute error")
                print("Mean: ", errors.mean())
                print("Std: ", errors.std())
                print()


            # Calibration of the ood detector
            pre_ood_score = martingale(compute_p_values(errors))   
            ood_score = step * np.log(1 + pre_ood_score) #/nb_steps
            #print("corrected score ", np.log10(ood_score)/nb_steps)
            if ood_score > threshold:
                return step, ood_score
        return nb_steps, ood_score

    def save(self, path):
        np.save(path / 'nonconformity_scores.npy', self.nonconformity_scores)
        
    def load(self, path):
        self.nonconformity_scores = np.load(path / 'nonconformity_scores.npy')



ood_detector = MartingaleOODDetector(original_env)

In [ ]:
Y = [ood_detector.test_ood(original_env, nb_steps=n) for n in range(1,1000)]
plt.plot(Y)

In [ ]:
threshold

In [ ]:
ood_detector.stop_above_threshold(original_env, threshold, start_at=50, nb_steps=100)

In [ ]:
ood_detector.stop_above_threshold(original_env, threshold, start_at=50, nb_steps=1000)

In [ ]:
default_values, values = get_mountain_car_values()
mean_ood_stops = {}
std_ood_stops = {}

original_env = instanciate_mountain_car(default_values)
agent = A2C.load(path/'Agents'/'trained_mountain_car_10000', env=original_env)
ood_detector = MartingaleOODDetector(env)
in_distrib_score = ood_detector.get_in_distrib_score()

for config in tqdm(ood_configs):
    if config['change'] not in mean_ood_scores:
        mean_ood_scores[config['change']] = []
        std_ood_scores[config['change']] = []
    
    ood_env = instanciate_mountain_car(config)
    agent = A2C.load(path/'Agents'/'trained_mountain_car_10000', env=ood_env)
    list_stops = np.array([ood_detector.stop_above_threshold(original_env, threshold, start_at=30, nb_steps=1000)[0] for _ in range(10)])
    mean_ood_stop = list_stops.mean()
    std_ood_stop = list_stops.std()

    #mean_reward, std_reward = evaluate(ood_env, agent, nb_episodes=10)

    mean_ood_stops[config['change']].append(mean_ood_stop)
    std_ood_stops[config['change']].append(std_ood_stop)
    #print(config, mean_reward)
    



In [ ]:
from turtle import color
from numpy import inf


for key in tqdm(results):
    fig, ax = plt.subplots(figsize=(10, 4), dpi=200)
    ax.set_xscale('log')
    ax.set_yscale('log')
    
    labels = copy(values[key])
    labels.append(default_values[key])
    labels = sorted(labels)
    
    #mean_ood_scores[key] = np.nan_to_num(mean_ood_scores[key], copy=True, nan=0.0, posinf=max(mean_ood_scores[key]), neginf=0)
    #std_ood_scores[key] = np.nan_to_num(std_ood_scores[key], copy=True, nan=0.0, posinf=max(std_ood_scores[key]), neginf=0)

    default_width = default_values[key]/6.0
    width = np.array([2**i for i in range(-5, 0)] + [2**i for i in range(1, 5+1)]) #np.array(labels[1:]) - np.array(labels[:-1]) 
    width *= default_width
    #print(width, np.array([width[-1]]))
    #width = np.concatenate((width, np.array([width[-1]])))/ 5
    #print(width)

    ax.bar(default_values[key]- default_width/2, 1.0, width=default_width, label='Mean stop', color='darkgreen')
    ax.bar(values[key]- width/2, results[key], width=width,  color='green')

    ax.bar(default_values[key]+default_width/2, 1000.0, width=default_width, label='OOD stop', color='darkorange')
    ax.bar(values[key]+ width/2, mean_ood_stops[key], width=width, color='orange', yerr=std_ood_scores[key])
    
    ax.set_xlabel(key)
    ax.set_ylabel('Mean episode duration')
    ax.set_title('Impact of OOD regarding '+key)

    plt.legend() #prop={'size': 'medium'}, ncol=2)
   

# Computation of the AUC score

In [ ]:
def compute_AUC(mean_ood_scores, plot=True, verbose=False):
    TPRs = []
    FPRs = []

    thresholds = [10**(-i/10) for i in range(-50, 50+1)]
    for threshold in thresholds:
        false_pos=0
        false_neg=0
        true_pos=0
        true_neg=0

        for key in mean_ood_scores:
            if str(key) != 'None': # Env OOD
                for test_res in mean_ood_scores[key]:

                    if test_res<threshold: # Not detected as OOD
                        false_neg +=1
                    else:
                        true_pos +=1
            else: # Env standard
                for test_res in mean_ood_scores[key]:

                    if test_res>threshold: # Detected as OOD
                        false_pos +=1
                    else:
                        true_neg +=1


        try:
            tpr = true_pos/(true_pos+false_neg)
        except ZeroDivisionError:
            print('error computing tpr')
            continue
            tpr = 1.0
        
        try:
            fpr = false_pos/(false_pos+true_neg)
        except ZeroDivisionError:
            print('error computing fpr')
            continue

        if verbose:
            print(threshold)
            print('TP', true_pos, 'FP',false_pos)
            print('FN', false_neg, 'TN',true_neg)
            print()
            print(tpr, fpr)
        TPRs.append(tpr)
        FPRs.append(fpr)
        #AUC += tpr/len(thresholds)

    if verbose:
        plt.title("TPR and FPR curves")
        plt.plot(TPRs, label='TPR')
        plt.plot(FPRs, label='FPR')
        plt.legend()
        plt.show()
        
    #print('AUC: ', AUC)
    plt.title("ROC curve")
    plt.plot(FPRs, TPRs,'-*')
    plt.xlabel('TPR')
    plt.ylabel('FPR')
    plt.show()

    AUC = integrate.simps(x=FPRs, y=TPRs, even='avg')
    return AUC

In [ ]:
compute_AUC(mean_ood_scores)